In [ ]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.text import text_to_word_sequence
import tensorflow as tf
import logging
tf.get_logger().setLevel(logging.ERROR)

EPOCHS = 32
BATCH_SIZE = 256
INPUT_FILE_NAME = '/content/drive/MyDrive/Software, AI, Distsys.../Deep Learning NVIDIA Book/frankenstein.txt'
WINDOW_LENGTH = 40
WINDOW_STEP = 3
PREDICT_LENGTH = 3
MAX_WORDS = 10000
EMBEDDING_WIDTH = 100

In [ ]:
# Open and read file
file = open(INPUT_FILE_NAME, 'r', encoding='utf-8')
text = file.read()
file.close()

# Make lower case and split into individual words
text = text_to_word_sequence(text)

# Create training example
fragments = []
targets = []
for i in range(0, len(text) - WINDOW_LENGTH, WINDOW_STEP):
  fragments.append(text[i: i + WINDOW_LENGTH])
  targets.append(text[i + WINDOW_LENGTH])

In [ ]:
# Convert to indices
tokenizer = Tokenizer(num_words=MAX_WORDS, oov_token='UNK')
tokenizer.fit_on_texts(text)
fragments_indexed = tokenizer.texts_to_sequences(fragments)
targets_indexed = tokenizer.texts_to_sequences(targets)

# Convert the appropriate input and output formats
x = np.array(fragments_indexed, dtype=np.int)
y = np.zeros((len(targets_indexed), MAX_WORDS))
for i, target_index in enumerate(targets_indexed):
  y[i, target_index] = 1

In [ ]:
# Build and train model
training_model = Sequential()
training_model.add(Embedding(
    output_dim=EMBEDDING_WIDTH, input_dim=MAX_WORDS,
    mask_zero=True, input_length=None))
training_model.add(LSTM(128, return_sequences=True,
                        dropout=0.2, recurrent_dropout=0.2))
training_model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
training_model.add(Dense(128, activation='relu'))
training_model.add(Dense(MAX_WORDS, activation='softmax'))
training_model.compile(loss='categorical_crossentropy', optimizer='adam')
training_model.summary()
history = training_model.fit(x, y, validation_split=0.05,
                             batch_size=BATCH_SIZE, epochs=EPOCHS,
                             verbose=2, shuffle=True)

In [ ]:
# Build stateful model used for prediction
inference_model = Sequential()
inference_model.add(Embedding(
    output_dim=EMBEDDING_WIDTH, input_dim=MAX_WORDS,
    mask_zero=True, batch_input_shape=(1, 1)))
inference_model.add(LSTM(128, return_sequences=True, dropout=0.2,
                         recurrent_dropout=0.2, stateful=True))
inference_model.add(LSTM(128, dropout=0.2,
                         recurrent_dropout=0.2, stateful=True))
inference_model.add(Dense(128, activation='relu'))
inference_model.add(Dense(MAX_WORDS, activation='softmax'))
weights = training_model.get_weights()
inference_model.set_weights(weights)

In [ ]:
# Provide beginning of sentence and
# predict next words in a greedy manner

first_words = ['i', 'saw']
first_words_indexed = tokenizer.texts_to_sequences(first_words)
inference_model.reset_states()
predicted_string = ''
# Feed initial words to the model
for i, word_index in enumerate(first_words_indexed):
  x = np.zeros((1,1), dtype=np.int)
  x[0][0] = word_index[0]
  predicted_string += first_words[i]
  predicted_string += ' '
  y_predict = inference_model.predict(x, verbose=0)[0]
# Predict PREDICT_LENGTH words.
for i in range(PREDICT_LENGTH):
  new_word_index = np.argmax(y_predict)
  word = tokenizer.sequences_to_texts([new_word_index])
  x[0][0] = new_word_index
  predicted_string += word[0]
  predicted_string += ' '
  y_predict = inference_model.predict(x, verbose=0)[0]
print(predicted_string)